In [1]:
# Cell ID: WnZAt57xrx5g
# Cell 1: Initial setup, connecting to Google Drive, installing libraries, and checking GPU availability.
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Instalar y actualizar las librerías
print("\nInstalando y actualizando librerías...")
!pip install --upgrade -q mne pytorch-lightning timm
print("✅ Librerías listas.")

# Step 3: Prueba explícita de control de la GPU
import torch
print("\n--- INICIANDO PRUEBA DE CONTROL DE GPU ---")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ GPU detectada: {torch.cuda.get_device_name(0)}")
    try:
        tensor_grande = torch.randn(1024, 1024, 512, device=device) # Asignar 2GB
        memoria_asignada = torch.cuda.memory_allocated(0) / 1024**3
        print(f"✅ ¡Éxito! Memoria asignada activamente: {memoria_asignada:.2f} GB")
        del tensor_grande
        torch.cuda.empty_cache()
        print("✅ Memoria liberada correctamente.")
        print("--- PRUEBA DE CONTROL DE GPU COMPLETADA EXITOSAMENTE ---")
    except Exception as e:
        print(f"❌ ¡ERROR DURANTE LA PRUEBA! No se pudo asignar memoria a la GPU: {e}")
else:
    print("❌ ¡ERROR! No se detectó ninguna GPU en este entorno de ejecución.")

Mounted at /content/drive

Instalando y actualizando librerías...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.2/828.2 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 65.8 MB/s eta 0:00:00
✅ Librerías listas.

--- INICIANDO PRUEBA DE CONTROL DE GPU ---
✅ GPU detectada: NVIDIA A100-SXM4-40GB
✅ ¡Éxito! Memoria asignada activamente: 2.00 GB
✅ Memoria liberada correctamente.
--- PRUEBA DE CONTROL DE GPU COMPLETADA EXITOSAMENTE ---


In [3]:
# Repeat test with 50 subjects, and three separate tests to find the best one (1e-5 (your current best), 5e-5, and 1e-4)
# ==============================================================================
# 1. SETUP AND DEPENDENCY INSTALLATION
# ==============================================================================
print("Ensuring PyTorch Lightning and other libraries are installed...")
# Install the necessary libraries with pinned versions to avoid conflicts
!pip install --upgrade -q pytorch-lightning timm "pandas==2.2.2" "pyarrow==19.0.0"
print("✅ Installation check complete.")

# ==============================================================================
# 2. IMPORTS AND INITIAL CONFIGURATION
# ==============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score
import numpy as np
import pandas as pd
from pathlib import Path
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set matrix multiplication precision for A100/H100 GPUs for better performance
torch.set_float32_matmul_precision('medium')
print("✅ Libraries imported and configuration set.")

# ==============================================================================
# 3. MODEL ARCHITECTURE DEFINITION
# ==============================================================================
def get_convnext_model(num_classes=5, pretrained=True):
    """
    Creates a ConvNeXT V2 model adapted for sleep stage classification.

    This function takes a pre-trained ConvNeXT V2 model and modifies it for our
    specific use case by:
    1. Changing the first convolutional layer to accept single-channel (grayscale)
       spectrogram images instead of the standard three-channel (RGB) images.
    2. Replacing the final classification layer to output scores for our
       `num_classes` sleep stages.

    Args:
        num_classes (int): The number of sleep stages to classify (e.g., 5).
        pretrained (bool): Whether to load weights pre-trained on ImageNet.

    Returns:
        A PyTorch nn.Module representing the adapted model.
    """
    # Load the specified ConvNeXT V2 model from the timm library
    model = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=pretrained)

    # --- Adapt the first convolutional layer for 1-channel input ---
    original_conv = model.stem[0]
    new_first_conv = nn.Conv2d(
        in_channels=1,
        out_channels=original_conv.out_channels,
        kernel_size=original_conv.kernel_size,
        stride=original_conv.stride,
        padding=original_conv.padding,
        bias=(original_conv.bias is not None)
    )

    # Copy the pre-trained weights, averaging them across the original input channels (RGB)
    # This is a common technique to transfer knowledge from RGB to grayscale.
    with torch.no_grad():
        if original_conv.weight.shape[1] == 3: # Check if original was 3-channel
            new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)
        else: # Handle cases where the pre-trained model isn't 3-channel
            print(f"Warning: Original model stem has {original_conv.weight.shape[1]} input channels. Initializing new conv weights randomly.")
            nn.init.kaiming_normal_(new_first_conv.weight, mode='fan_out', nonlinearity='relu')

    model.stem[0] = new_first_conv

    # --- Adapt the final fully connected layer for our number of classes ---
    num_ftrs = model.head.fc.in_features
    model.head.fc = nn.Linear(num_ftrs, num_classes)

    return model

print("✅ `get_convnext_model` function defined.")

# ==============================================================================
# 4. PYTORCH LIGHTNING MODULE
# ==============================================================================
class SleepStageClassifierLightning(pl.LightningModule):
    """
    PyTorch Lightning module for sleep stage classification.

    This class encapsulates the entire model, training, validation, and optimization
    logic, making the training process clean and reproducible.
    """
    def __init__(self, learning_rate=1e-5, class_weights=None):
        super().__init__()
        # Save hyperparameters (like learning_rate) to the checkpoint
        self.save_hyperparameters()

        # Initialize the model using our custom function
        self.model = get_convnext_model(num_classes=5, pretrained=True)

        # Define metrics for tracking performance
        self.train_accuracy = MulticlassAccuracy(num_classes=5)
        self.val_accuracy = MulticlassAccuracy(num_classes=5)
        self.train_f1 = MulticlassF1Score(num_classes=5, average='macro')
        self.val_f1 = MulticlassF1Score(num_classes=5, average='macro')

        # Set up the loss function with optional class weights to handle data imbalance
        self.weights = torch.tensor(class_weights, dtype=torch.float) if class_weights is not None else None
        self.loss_fn = nn.CrossEntropyLoss(weight=self.weights)

    def forward(self, x):
        """Defines the forward pass of the model."""
        return self.model(x)

    def training_step(self, batch, batch_idx):
        """Logic for a single training step."""
        x, y_true = batch
        y_pred_logits = self(x)
        loss = self.loss_fn(y_pred_logits, y_true)

        # Log metrics for this step
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', self.train_accuracy(y_pred_logits, y_true), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_f1', self.train_f1(y_pred_logits, y_true), on_step=False, on_epoch=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        """Logic for a single validation step."""
        x, y_true = batch
        y_pred_logits = self(x)
        loss = self.loss_fn(y_pred_logits, y_true)

        # Log metrics for this step
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', self.val_accuracy(y_pred_logits, y_true), on_epoch=True, prog_bar=True, logger=True)
        self.log('val_f1', self.val_f1(y_pred_logits, y_true), on_epoch=True, logger=True)

        return loss

    def configure_optimizers(self):
        """Set up the optimizer and learning rate scheduler."""
        # AdamW is a robust optimizer that often works well with transformer-based models
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)

        # The scheduler reduces the learning rate when validation loss plateaus
        # NOTE: The `verbose` argument was removed as it's deprecated in recent PyTorch versions.
        scheduler = {
            'scheduler': ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3),
            'monitor': 'val_loss', # Metric to monitor
            'interval': 'epoch',
            'frequency': 1,
        }
        return [optimizer], [scheduler]

print("✅ `SleepStageClassifierLightning` module defined.")

# ==============================================================================
# 5. CUSTOM DATASET DEFINITION
# ==============================================================================
class CombinedDataset(Dataset):
    """
    Custom dataset to efficiently load sleep spectrograms from a list of Parquet files.
    """
    def __init__(self, file_paths_chunk):
        print(f"Initializing dataset with {len(file_paths_chunk)} files...")
        self.file_paths = file_paths_chunk
        self.epochs_per_file = []
        self._cache = {} # Cache to hold dataframes in memory

        # Pre-calculate the number of valid samples in each file
        for f_path in self.file_paths:
            try:
                df_labels = pd.read_parquet(f_path, columns=['label'])
                # Count only the labels we are interested in (0-4)
                num_valid = df_labels['label'].isin([0, 1, 2, 3, 4]).sum()
                self.epochs_per_file.append(num_valid)
            except Exception as e:
                print(f"Warning: Could not read or process {f_path.name}. Skipping. Error: {e}")
                self.epochs_per_file.append(0)

        # Create a cumulative sum to easily map a global index to a file and local index
        self.cumulative_epochs = np.cumsum(self.epochs_per_file)
        self.total_epochs = self.cumulative_epochs[-1] if len(self.cumulative_epochs) > 0 else 0
        print(f"✅ Dataset initialized. Total valid epochs: {self.total_epochs}")

    def __len__(self):
        """Returns the total number of valid samples across all files."""
        return self.total_epochs

    def __getitem__(self, idx):
        """
        Retrieves a single sample (spectrogram and label) from the dataset.
        """
        # Find which file this index belongs to
        file_idx = np.searchsorted(self.cumulative_epochs, idx, side='right')

        # Calculate the local index within that file
        local_idx = idx - (self.cumulative_epochs[file_idx - 1] if file_idx > 0 else 0)

        file_path = self.file_paths[file_idx]

        # Load dataframe from cache or disk
        if file_path not in self._cache:
            try:
                df = pd.read_parquet(file_path)
                # Filter for valid labels and reset index
                self._cache[file_path] = df[df['label'].isin([0, 1, 2, 3, 4])].reset_index(drop=True)
            except Exception as e:
                raise IOError(f"Error reading file {file_path.name} in __getitem__: {e}")

        # Retrieve the specific row
        row = self._cache[file_path].iloc[local_idx]
        label = np.int64(row['label'])
        spectrogram_flat = row.drop('label').values.astype(np.float32)

        # Z-score normalization (per sample)
        mean, std = spectrogram_flat.mean(), spectrogram_flat.std()
        spectrogram_normalized = (spectrogram_flat - mean) / (std + 1e-6) # Add epsilon to avoid division by zero

        # Reshape to [Channel, Height, Width]
        spectrogram_2d = spectrogram_normalized.reshape(1, 76, 60)

        return torch.from_numpy(spectrogram_2d), torch.tensor(label)

print("✅ `CombinedDataset` class defined.")

# ==============================================================================
# 6. TRAINING EXECUTION
# ==============================================================================
print("\n--- Starting Learning Rate Tuning Experiment ---")

# --- General Parameters ---
EPOCHS = 40
BATCH_SIZE = 256
NUM_WORKERS = 8
CLASS_WEIGHTS = [0.7, 3.5, 0.5, 1.5, 1.2]
LEARNING_RATES_TO_TEST = [1e-5, 5e-5, 1e-4] # List of LRs to experiment with

# --- Paths and File Identification ---
shhs1_processed_dir_base = Path('/content/drive/MyDrive/shhs1_processed')
shhs2_processed_dir_base = Path('/content/drive/MyDrive/shhs2_processed')

specific_shhs_subjects = [
    # 25 files from SHHS1
    'shhs1-200001.parquet', 'shhs1-200002.parquet', 'shhs1-200003.parquet',
    'shhs1-200004.parquet', 'shhs1-200005.parquet', 'shhs1-200006.parquet',
    'shhs1-200007.parquet', 'shhs1-200008.parquet', 'shhs1-200009.parquet',
    'shhs1-200010.parquet', 'shhs1-200011.parquet', 'shhs1-200012.parquet',
    'shhs1-200013.parquet', 'shhs1-200014.parquet', 'shhs1-200015.parquet',
    'shhs1-200016.parquet', 'shhs1-200017.parquet', 'shhs1-200018.parquet',
    'shhs1-200019.parquet', 'shhs1-200020.parquet', 'shhs1-200021.parquet',
    'shhs1-200022.parquet', 'shhs1-200023.parquet', 'shhs1-200024.parquet',
    'shhs1-200025.parquet',
    # 25 files from SHHS2
    'shhs2-200077.parquet', 'shhs2-200078.parquet', 'shhs2-200079.parquet',
    'shhs2-200080.parquet', 'shhs2-200081.parquet', 'shhs2-200082.parquet',
    'shhs2-200086.parquet', 'shhs2-200088.parquet', 'shhs2-200089.parquet',
    'shhs2-200091.parquet', 'shhs2-200092.parquet', 'shhs2-200093.parquet',
    'shhs2-200102.parquet', 'shhs2-200103.parquet', 'shhs2-200105.parquet',
    'shhs2-200107.parquet', 'shhs2-200108.parquet', 'shhs2-200109.parquet',
    'shhs2-200110.parquet', 'shhs2-200111.parquet', 'shhs2-200112.parquet',
    'shhs2-200115.parquet', 'shhs2-200116.parquet', 'shhs2-200117.parquet',
    'shhs2-200121.parquet',
]

specific_shhs_file_paths = []
for filename in specific_shhs_subjects:
    if filename.startswith('shhs1'):
        full_path = shhs1_processed_dir_base / filename
    elif filename.startswith('shhs2'):
        full_path = shhs2_processed_dir_base / filename
    else:
        print(f"Warning: Filename format not recognized: {filename}. Skipping.")
        continue

    if full_path.exists():
        specific_shhs_file_paths.append(full_path)
    else:
        print(f"Warning: File not found at {full_path}. Skipping.")

# --- Main Experiment Loop ---
if not specific_shhs_file_paths:
     print("\nERROR: No valid .parquet files were found. Aborting experiment.")
else:
    print(f"\nFound {len(specific_shhs_file_paths)} specific files for training.")

    # --- Create Dataset and Dataloaders (once, as they are the same for each LR) ---
    full_dataset = CombinedDataset(specific_shhs_file_paths)

    if len(full_dataset) > 1:
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

        print(f"Dataset split: {len(train_dataset)} training samples, {len(val_dataset)} validation samples.")

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=True)

        # --- Loop over each learning rate ---
        for lr in LEARNING_RATES_TO_TEST:
            print(f"\n{'='*20} STARTING EXPERIMENT FOR LR = {lr} {'='*20}")

            # --- Initialize Model, Logger, and Callbacks for this run ---
            model = SleepStageClassifierLightning(learning_rate=lr, class_weights=CLASS_WEIGHTS)

            # Use a unique name for logger and checkpoint for each LR
            lr_str = f"{lr:.0e}".replace('-', '') # Format LR for filenames (e.g., 1e-05 -> 1e05)
            experiment_name = f"convnext_50_files_40_epochs_lr_{lr_str}"

            csv_logger = CSVLogger("/content/drive/MyDrive/sleep_logs/", name=experiment_name)

            checkpoint_callback = ModelCheckpoint(
                monitor='val_loss',
                dirpath='/content/drive/MyDrive/final_model_checkpoint/',
                filename=f"sleep-stage-model-{experiment_name}-{{epoch:02d}}-{{val_loss:.4f}}",
                save_top_k=1,
                mode='min'
            )

            # --- Initialize and Run Trainer ---
            trainer = pl.Trainer(
                max_epochs=EPOCHS,
                accelerator="gpu",
                devices=1,
                logger=csv_logger,
                callbacks=[checkpoint_callback],
                precision="bf16-mixed",
                gradient_clip_val=1.0
            )

            print(f"🚀 Starting model training for LR = {lr}...")
            trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
            print(f"✅ Training complete for LR = {lr}!")
            print(f"Best model for this run saved at: {checkpoint_callback.best_model_path}")
            print(f"{'='*20} FINISHED EXPERIMENT FOR LR = {lr} {'='*20}")

    else:
        print("Dataset is too small to split. Aborting experiment.")

print("\n--- All Learning Rate Experiments Complete ---")

Ensuring PyTorch Lightning and other libraries are installed...
✅ Installation check complete.
✅ Libraries imported and configuration set.
✅ `get_convnext_model` function defined.
✅ `SleepStageClassifierLightning` module defined.
✅ `CombinedDataset` class defined.

--- Starting Learning Rate Tuning Experiment ---

Found 50 specific files for training.
Initializing dataset with 50 files...
✅ Dataset initialized. Total valid epochs: 56638
Dataset split: 45310 training samples, 11328 validation samples.

==================== STARTING EXPERIMENT FOR LR = 1e-05 ====================


INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | ConvNeXt           | 27.9 M | train
1 | train_accuracy | MulticlassAccuracy | 0      | train
2 | val_accuracy   | MulticlassAccuracy | 0      | train
3 | train_f1       | MulticlassF1Score  | 0      | train
4 | val_f1         | MulticlassF1Score  | 0      | train
5 | loss_fn        | CrossEntropyLoss   | 0      | train
--------------------------------------------------------------
27.9 M   

🚀 Starting model training for LR = 1e-05...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=40` reached.


✅ Training complete for LR = 0.0001!
Best model for this run saved at: /content/drive/MyDrive/final_model_checkpoint/sleep-stage-model-convnext_50_files_40_epochs_lr_1e04-epoch=08-val_loss=0.6697.ckpt
==================== FINISHED EXPERIMENT FOR LR = 0.0001 ====================

--- All Learning Rate Experiments Complete ---
